# EEG Eye State detection

## Authors: Ángel López Manríquez

### Objectives:
### What is done in the Notebook:

In [22]:

#import arff

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


from IPython.display import display, HTML, Markdown

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score

# classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC  # Support vector classifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from matplotlib.colors import ListedColormap


In [23]:
plt.style.use('seaborn')

## Reading the dataset

In [9]:
# A VER PRUEBA ESTOOOO
from scipy.io import arff
import pandas as pd

filename = 'EEG Eye State.arff'
data = arff.loadarff(filename)
df = pd.DataFrame(data[0])

dataset=df

print('type: ', type(dataset))
print('attributes: ', dataset.keys())
print(df)

#print('description: ', dataset['description'], '\n')
#print('relation: ', dataset['relation'], '\n')
#print('attributes: ', dataset['attributes'], '\n')
#print('data (first 5 samples): ', '\n')




type:  <class 'pandas.core.frame.DataFrame'>
attributes:  Index(['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6',
       'F4', 'F8', 'AF4', 'eyeDetection'],
      dtype='object')
           AF3       F7       F3      FC5       T7       P7       O1       O2  \
0      4329.23  4009.23  4289.23  4148.21  4350.26  4586.15  4096.92  4641.03   
1      4324.62  4004.62  4293.85  4148.72  4342.05  4586.67  4097.44  4638.97   
2      4327.69  4006.67  4295.38  4156.41  4336.92  4583.59  4096.92  4630.26   
3      4328.72  4011.79  4296.41  4155.90  4343.59  4582.56  4097.44  4630.77   
4      4326.15  4011.79  4292.31  4151.28  4347.69  4586.67  4095.90  4627.69   
5      4321.03  4004.62  4284.10  4153.33  4345.64  4587.18  4093.33  4616.92   
6      4319.49  4001.03  4280.51  4151.79  4343.59  4584.62  4089.74  4615.90   
7      4325.64  4006.67  4278.46  4143.08  4344.10  4583.08  4087.18  4614.87   
8      4326.15  4010.77  4276.41  4139.49  4345.13  4584.10  4091.28  46

In [10]:

filename = 'EEG Eye State.arff'
with open(filename) as file:  # no mode='rb' supplied
    dataset = arff.load(file)

    print('type: ', type(dataset))
    print('attributes: ', dataset.keys())

    print('description: ', dataset['description'], '\n')
    print('relation: ', dataset['relation'], '\n')
    print('attributes: ', dataset['attributes'], '\n')
    print('data (first 5 samples): ', '\n')

    pd.DataFrame(dataset['data'][:5], columns=[x[0] for x in dataset['attributes']])

AttributeError: module 'scipy.io.arff' has no attribute 'load'

## Preprocessing the data

In [6]:
# Set each data[:][-1] as int instead of str. 
# If there's one str in the list passed to np.array ALL the elements in np.array will be of type str
data = dataset['data']
m, n = len(data), len(data[0])
for i, row in enumerate(data):
    data[i][n - 1] = int(row[n - 1]) 

# Another way to perform the previous task
# data = np.array(dataset['data']).astype('float64')
# data

KeyError: 'data'

In [44]:
data = np.array(data)
X = data[:, :-1]
y = data[:, -1]

# sc = StandardScaler()
# X = sc.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

## Testing the classifiers
We'll determine which is the most suitable model for our dataset.

In [27]:
def display_markdown(txt):
    display(Markdown(txt))

def classifier_summary(clf, name, X_train, X_test, y_train, y_test, latex=False):
    '''clf: Classifier '''
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    display_markdown(f'## {name}')
    print(accuracy_score(y_test, y_pred))
    cm = pd.crosstab(y_test, y_pred)
    if latex: print(cm.to_latex())
    display(cm)
    
    

In [28]:

lg = LogisticRegression(random_state=0)
gnb = GaussianNB()
knc = KNeighborsClassifier(n_neighbors=3)
dtc = DecisionTreeClassifier(random_state=0)
rfc = RandomForestClassifier(random_state=0)


In [29]:
for clf, name in ((lg,'Logistic "Regression"'), (gnb,'Naive bayes (Gaussian)'), 
                  (knc,'KNeighbors classifier'), 
                  (dtc,'Decision Tree Classifier'), (rfc, 'Random forest classifier'), ):
    classifier_summary(clf, name, X_train, X_test, y_train, y_test)              


/home/ang3lino/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


## Logistic "Regression"

0.6401869158878505


col_0,0.0,1.0
row_0,,
0.0,1253,372
1.0,706,665


## Naive bayes (Gaussian)

...

col_0,0.0,1.0
row_0,,
0.0,1360,265
1.0,292,1079


/home/ang3lino/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


## Random forest classifier

0.8945260347129506


col_0,0.0,1.0
row_0,,
0.0,1545,80
1.0,236,1135
